## Kotlin의 Null 안전성 정리

### 1. Kotlin의 Null 처리 철학
- Kotlin은 Java에서 흔히 발생하는 NullPointerException (NPE) 를 컴파일 단계에서 원천적으로 방지하고자 설계됨.
- Kotlin에서 모든 변수는 기본적으로 null이 될 수 없음.
- null을 허용하려면 명시적으로 타입에 ?를 붙여야 함.

In [ ]:
val str: String = null // 컴파일에러
val str: String? = null //가능

### 2. Nullable 타입과 사용 제한
- String?은 null을 허용하지만, 사용 시 제약이 생김
- Nullable 변수에서는 일반적인 메서드나 프로퍼티 사용이 제한됨

In [ ]:
val str: String? = "Hello"
val upper = str.uppercase() //null일 수 있음

### 3. Null-Safe 접근: Safe call operator(?.)
- ?.는 객체가 null이 아닐 때만 메서드나 프로퍼티에 접근함
- 객체가 null이면 전체 표현식은 null로 평가됨

In [1]:
val str: String? = "hello"
println(str?.uppercase())

val str2: String? = null
println(str2?.uppercase())

HELLO
null


### 4. Elvis Operator(?:)

In [2]:
val str: String? = null
val result: String = str ?: "기본값"
println(result)

기본값


### 5. Safe Call + Elvis Operator 조합
- 여러 nullable 객체를 연결할 때 중간에 하나라도 null이면 전체가 null로 평가됨
- 마지막에 Elvis 연산자를 써서 기본값을 줄 수 있음'

In [ ]:
val countryCode: String = bankBranch?.address?.country?.countryCode ?: "US"

### 6. Safe Cast (as?)
- 타입 캐스팅 시 실패해도 예외가 발생하지 않고 null을 반환

In [4]:
val some: Any = arrayOf(1, 2, 3)
val str: String? = some as? String
println(str)
println(str?.uppercase())

null
null


### 7. Smart Cast와 컴파일러 추론
- if (str != null)로 null 체크 후에는 컴파일러가 non-null로 간주하고 일반 메서드 사용 가능

In [6]:
val str: String? = "heel"
if (str != null) {
    println(str.uppercase()) // 안전하게 호출됨
}

println(str?.uppercase())

HEEL
HEEL


## Kotlin의 Null 처리: 고급 개념과 실용적 고려사항

### 1. Not-null assertion (!!)
- 절대 null이 아닐 것이라 "확신"할 때
- null일 경우 즉시 예외 발생 (KotlinNullPointerException)
- null이 아니라면 non-null 타입처럼 사용 가능
- **null이 아니라고 100% 확신할 때만 사용**해야 하며, 실제론 그 확신이 종종 틀립니다.

In [9]:
val str: String? = "hello"

val upper = str!!.uppercase()
val str2: String = str!!
println(str2)

val str3: String? = null
val str4: String = str3!!  // 런타임에 KotlinNullPointerException 발생

hello


java.lang.NullPointerException: 

### 2. !! 사용의 치명적인 단점
- 예외가 어디서 발생했는지 알기 어려움
  - 여러 !!를 체인처럼 연결할 경우 어느 부분이 null이었는지 디버깅 어려움
- 스택 트레이스에 정보 없음
  - 예외 메시지는 단순히 "null이었습니다"일 뿐, 어떤 표현식이 문제인지 알려주지 않음.

In [ ]:
val countryCode = bankBranch!!.address!!.country!!.countryCode

### 3. Nullable 타입을 Non-null 타입에 넘겨야 할 때

In [10]:
fun printText(text: String) {
    println(text)
}

val str: String? = "Hello"
printText(str) //오류: String? → String 불가

Hello
Hello


#### !! 사용

In [ ]:
val str: String? = "Hello"
printText(str!!) // 위험하지만 강제로 nullable → non-null

#### let함수 + safe call
- let 함수는 null이 아닐 경우에만 블록 실행
- null이면 아무 일도 안 함
- it은 블록 내에서 실제 non-null 값

**let 체이닝**
- 불필요한 중첩 if를 제거
- null-safe한 방식으로 처리
- 가독성 유지
- 중간에 하나라도 null이면 안전하게 종료

In [15]:
str4?.let { printText(it) }// 안전하게 처리됨

data class City(val name: String)
data class Address(val city: City)
data class User(val address: Address?)

fun printCityName(user: User?) {
    user?.address
        ?.let { it.city }
        ?.let { it.name }
        ?.let { println("City: $it") }

    user?.address?.city?.name?.run {
        println("City: $this") //바로앞에 name값을 의미
    }
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[15], line 1, column 5: Unnecessary safe call on a non-null receiver of type String
Line_26.jupyter.kts (17:36 - 19:6) Type mismatch: inferred type is String.(Any?) -> Unit but TypeVariable(T).() -> TypeVariable(R) was expected
at Cell In[15], line 17, column 38: Cannot infer a type for this parameter. Please specify it explicitly.
at Cell In[15], line 17, column 38: Expected no parameters 

### 4. equals 비교는 null-safe
- Kotlin에서 ==는 .equals() 호출을 의미하지만
- null 체크는 내부에서 자동으로 수행
- NullPointerException 발생 안 함

In [ ]:
val str1: String? = null
val str2: String = "Kotlin"

println(str1 == str2)  // ✅ null-safe 비교

### 5. Null을 명시적으로 체크해야 하는 경우
- Safe call 연산자(?.)로는 모든 값이 null이 아님을 동시에 체크할 수 없음.

In [ ]:
if (a != null && b != null && c != null) {
    // do something
}